In [382]:
# import libraries
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.model_selection import train_test_split

print(tf.__version__)

2.11.0


In [383]:
data = pd.read_csv('train_data.csv')
dataT = pd.read_csv('test_data.csv')

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29020 entries, 0 to 29019
Data columns (total 19 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   BookingID                29020 non-null  int64  
 1   LeadTime                 29020 non-null  int64  
 2   ArrivalYear              29020 non-null  int64  
 3   ArrivalMonth             29020 non-null  int64  
 4   ArrivalDate              29020 non-null  int64  
 5   NumWeekendNights         29020 non-null  int64  
 6   NumWeekNights            29020 non-null  int64  
 7   MealPlan                 29020 non-null  object 
 8   Parking                  29020 non-null  int64  
 9   RoomType                 29020 non-null  object 
 10  NumAdults                29020 non-null  int64  
 11  NumChildren              29020 non-null  int64  
 12  MarketSegment            29020 non-null  object 
 13  RepeatedGuest            29020 non-null  int64  
 14  NumPrevCancellations  

In [384]:
data['MealPlan'] = pd.factorize(data.MealPlan)[0]
data['RoomType'] = pd.factorize(data.RoomType)[0]
data['MarketSegment'] = pd.factorize(data.MarketSegment)[0]
data['BookingStatus'] = pd.factorize(data.BookingStatus)[0]

dataT['MealPlan'] = pd.factorize(dataT.MealPlan)[0]
dataT['RoomType'] = pd.factorize(dataT.RoomType)[0]
dataT['MarketSegment'] = pd.factorize(dataT.MarketSegment)[0]
dataT['BookingStatus'] = pd.factorize(dataT.BookingStatus)[0]

data

,BookingID,LeadTime,ArrivalYear,ArrivalMonth,ArrivalDate,NumWeekendNights,NumWeekNights,MealPlan,Parking,RoomType,NumAdults,NumChildren,MarketSegment,RepeatedGuest,NumPrevCancellations,NumPreviousNonCancelled,AvgRoomPrice,SpecialRequests,BookingStatus
0,1,10,2018,3,31,0,1,0,0,0,1,0,0,0,0,0,95.00,0,0
1,2,116,2018,2,28,2,1,0,0,0,1,0,1,0,0,0,61.00,0,0
2,3,11,2018,7,25,1,2,0,0,0,2,1,1,0,0,0,129.75,1,1
3,4,3,2017,9,12,0,1,0,0,0,2,0,1,0,0,0,152.00,3,1
4,5,28,2018,3,7,1,3,0,0,0,2,0,2,0,0,0,87.00,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29015,29016,2,2018,11,21,1,2,0,0,0,1,0,1,0,0,0,100.67,1,1
29016,29017,18,2018,4,12,0,2,2,0,0,2,0,1,0,0,0,119.00,1,1
29017,29018,8,2017,9,29,0,3,0,0,0,1,0,0,0,0,0,65.00,0,1
29018,29019,20,2018,9,18,2,2,0,0,0,1,0,2,0,0,0,90.00,0,0


In [385]:
springMonths = [3,4,5], 
summerMonths = [6,7,8],
fallMonths = [9,10,11],
winterMonths = [12, 1, 2]

data["Seasons"] = data["ArrivalMonth"].replace(springMonths, 13)
data["Seasons"] = data["Seasons"].replace(summerMonths, 14)
data["Seasons"] = data["Seasons"].replace(fallMonths, 15)
data["Seasons"] = data["Seasons"].replace(winterMonths, 16)


seasons = data.groupby(['Seasons', 'RoomType', 'MarketSegment']).agg({
                                                'BookingID': 'count',
                                                'BookingStatus': [lambda x: sum(x==0),lambda x: sum(x==1)]
                                            })
# seasons = data.groupby(['Seasons', 'MarketSegment', 'RoomType']).agg({
#                                                 'BookingID': 'count',
#                                                 'BookingStatus': [lambda x: sum(x==0),lambda x: sum(x==1)]
#                                             })


seasons= seasons.rename(columns={
                            'count': 'Total Booking',
                            '<lambda_0>': 'Total Canceled',
                            '<lambda_1>': 'Total Not Canceled'
                        })
seasons.to_csv('result\groupby.csv')

seasons.head(50)

BookingID  BookingStatus                   
                               Total Booking Total Canceled Total Not Canceled
Seasons RoomType MarketSegment                                                
13      0        0                       322             61                261
                 1                      2779            982               1797
                 2                      1352            460                892
                 3                        26              0                 26
                 4                        33             10                 23
        1        1                       175             95                 80
                 2                         2              0                  2
                 3                         2              0                  2
        2        0                        23              5                 18
                 1                      1145            431                714
                 2                        76              3                 73
                 3                        11              0                 11
                 4                        14             11                  3
        3        1                        92             32                 60
                 2                        16              5                 11
        4        0                         7              0                  7
                 1                        18              6                 12
                 2                        10              0                 10
                 3                         1              0                  1
        5        1                        13              6                  7
                 3                         8              0                  8
14      0        0                       231             27                204
                 1                      3733           1628               2105
                 2                      2054            805               1249
                 3                        46              0                 46
                 4                         7              1                  6
        1        1                       239            113                126
                 2                         2              2                  0
                 3                         1              0                  1
        2        0                        27              8                 19
                 1                      1280            594                686
                 2                        85             18                 67
                 3                         7              0                  7
        3        1                       156             77                 79
                 2                         9              0                  9
                 3                         5              0                  5
        4        0                        18              4                 14
                 1                        14              5                  9
                 2                        20              7                 13
                 3                         5              0                  5
        5        1                        38             14                 24
                 2                         1              1                  0
                 3                         7              0                  7
15      0        0                       491             36                455
                 1                      4382           1573               2809
                 2                      3310           1129               2181
                 3                        81              0                 81
                 4                         7              2                  5
      